In [47]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor
from model_datetime_utils import feature_create

In [48]:
def function(x):
    if x>=0 :
        return 1
    else:
        return -1

In [49]:
feature_df = pd.read_csv("NF_label.csv",encoding="gbk")
multi_label = pd.read_csv(r"NF_multilabel.csv")
multi_label["datetime"] = pd.to_datetime(multi_label["datetime"])
multi_label["datetime"] = [str(i) for i in multi_label["datetime"]]
if "datetime" in feature_df.columns.tolist():
    feature_df["datetime"] = pd.to_datetime(feature_df["datetime"])
    feature_df["datetime"] = [str(i) for i in feature_df["datetime"]]
else:
    feature_df["datetime"] = pd.to_datetime(feature_df["date"] + ' ' + feature_df["time"])
    feature_df = feature_df.drop(columns=["date","time"])
    feature_df["datetime"] = [str(i) for i in feature_df["datetime"]]
final_df = pd.merge(feature_df, multi_label, on="datetime", how="left")
final_df.index = final_df["datetime"]
final_df = final_df.drop(columns=["datetime"])

In [50]:
final_df

,US_NF_EC_MoM_Actual,US_NF_EC_MoM_Srv_Med,US_NF_EC_MoM_Prior,US_NF_EC_MoM_Revised,US_NF_UER_MoM_Actual,US_NF_UER_MoM_Srv_Med,US_NF_UER_MoM_Prior,US_NF_UER_MoM_Revised,US_NF_HWR_MoM_Actual,US_NF_HWR_MoM_Srv_Med,US_NF_HWR_MoM_Prior,US_NF_HWR_MoM_Revised,3s_1min,3s_3min,3s_5min,3s_15min,3s_30min,3s_60min
datetime,,,,,,,,,,,,,,,,,,
2023-05-05 20:30:00,253,236,236,165,3.4,3.5,3.5,3.5,0.5,0.3,0.3,0.3,0.570,0.273,2.840,-15.607,-13.787,-15.957
2023-03-10 21:30:00,311,205,517,504,3.6,3.4,3.4,3.4,0.2,0.3,0.3,0.3,3.220,0.616,3.726,7.709,6.299,13.269
2023-02-03 21:30:00,517,185,223,260,3.4,3.6,3.5,3.5,0.3,0.3,0.3,0.4,-1.450,-1.910,-8.142,-16.800,-18.180,-24.070
2023-01-06 21:30:00,223,200,263,256,3.5,3.7,3.7,3.6,0.3,0.4,0.6,0.4,-2.339,0.428,-0.409,7.391,11.361,5.491
2022-12-02 21:30:00,263,200,261,284,3.7,3.7,3.7,3.7,0.6,0.3,0.4,0.5,-3.880,-4.701,-2.890,-6.261,-4.241,-4.111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-06-04 20:30:00,431,500,290,290,9.7,9.8,9.9,9.9,0.3,0.1,0.0,0.1,0.287,3.120,4.554,-0.393,2.548,3.036
2010-05-07 20:30:00,290,187,162,230,9.9,9.7,9.7,9.7,0.0,0.2,-0.1,-0.1,0.091,0.904,1.456,1.584,1.739,6.210
2010-03-05 21:30:00,-36,-55,-20,-26,9.7,9.8,9.7,9.7,0.1,0.2,0.3,0.3,-1.670,-0.994,-3.509,-2.015,-0.569,0.396


In [51]:
final_df["3s_1min"]=final_df["3s_1min"].apply(lambda  x : function(x))
final_df["3s_3min"]=final_df["3s_3min"].apply(lambda  x : function(x))
final_df["3s_5min"]=final_df["3s_5min"].apply(lambda  x : function(x))
final_df["3s_15min"]=final_df["3s_15min"].apply(lambda  x : function(x))
final_df["3s_30min"]=final_df["3s_30min"].apply(lambda  x : function(x))
final_df["3s_60min"]=final_df["3s_60min"].apply(lambda  x : function(x))


# 可选参数

In [52]:
model_name = "NF"  # 模型名称，与文件夹一致
fence = 15  # 测试集数量，一般为2022和2023作为测试集
HoldingTime = '30000'  # 回测模板中的持仓时间，单位为毫秒
trading_pair = "XAU/USD"

# 导入数据

In [53]:
df = final_df
df.head()

,US_NF_EC_MoM_Actual,US_NF_EC_MoM_Srv_Med,US_NF_EC_MoM_Prior,US_NF_EC_MoM_Revised,US_NF_UER_MoM_Actual,US_NF_UER_MoM_Srv_Med,US_NF_UER_MoM_Prior,US_NF_UER_MoM_Revised,US_NF_HWR_MoM_Actual,US_NF_HWR_MoM_Srv_Med,US_NF_HWR_MoM_Prior,US_NF_HWR_MoM_Revised,3s_1min,3s_3min,3s_5min,3s_15min,3s_30min,3s_60min
datetime,,,,,,,,,,,,,,,,,,
2023-05-05 20:30:00,253,236,236,165,3.4,3.5,3.5,3.5,0.5,0.3,0.3,0.3,1,1,1,-1,-1,-1
2023-03-10 21:30:00,311,205,517,504,3.6,3.4,3.4,3.4,0.2,0.3,0.3,0.3,1,1,1,1,1,1
2023-02-03 21:30:00,517,185,223,260,3.4,3.6,3.5,3.5,0.3,0.3,0.3,0.4,-1,-1,-1,-1,-1,-1
2023-01-06 21:30:00,223,200,263,256,3.5,3.7,3.7,3.6,0.3,0.4,0.6,0.4,-1,1,-1,1,1,1
2022-12-02 21:30:00,263,200,261,284,3.7,3.7,3.7,3.7,0.6,0.3,0.4,0.5,-1,-1,-1,-1,-1,-1


In [54]:
df = df.dropna(axis=0)
df.shape

(155, 18)

In [55]:
onnx_test = df[:fence].drop(columns=["3s_1min", "3s_3min", "3s_5min", "3s_15min", "3s_30min", "3s_60min"])
print(onnx_test.shape)
print(onnx_test.head())

(15, 12)
                     US_NF_EC_MoM_Actual  US_NF_EC_MoM_Srv_Med  \
datetime                                                         
2023-05-05 20:30:00                  253                   236   
2023-03-10 21:30:00                  311                   205   
2023-02-03 21:30:00                  517                   185   
2023-01-06 21:30:00                  223                   200   
2022-12-02 21:30:00                  263                   200   

                     US_NF_EC_MoM_Prior  US_NF_EC_MoM_Revised  \
datetime                                                        
2023-05-05 20:30:00                 236                   165   
2023-03-10 21:30:00                 517                   504   
2023-02-03 21:30:00                 223                   260   
2023-01-06 21:30:00                 263                   256   
2022-12-02 21:30:00                 261                   284   

                     US_NF_UER_MoM_Actual  US_NF_UER_MoM_Srv_Med  \
date

# 特征工程

In [56]:
feature_create('US_NF_EC_MoM', df)
feature_create('US_NF_UER', df)
feature_create('US_NF_HWR_MoM', df)

In [57]:
df.columns

Index(['US_NF_EC_MoM_Actual', 'US_NF_EC_MoM_Srv_Med', 'US_NF_EC_MoM_Prior',
       'US_NF_EC_MoM_Revised', 'US_NF_UER_MoM_Actual', 'US_NF_UER_MoM_Srv_Med',
       'US_NF_UER_MoM_Prior', 'US_NF_UER_MoM_Revised', 'US_NF_HWR_MoM_Actual',
       'US_NF_HWR_MoM_Srv_Med', 'US_NF_HWR_MoM_Prior', 'US_NF_HWR_MoM_Revised',
       '3s_1min', '3s_3min', '3s_5min', '3s_15min', '3s_30min',
       'US_NF_EC_MoM_Actual_sub_US_NF_EC_MoM_Srv_Med',
       'US_NF_EC_MoM_Actual_sub_US_NF_EC_MoM_Revised',
       'US_NF_EC_MoM_Actual_sub_US_NF_EC_MoM_Prior',
       'US_NF_HWR_MoM_Actual_sub_US_NF_HWR_MoM_Srv_Med',
       'US_NF_HWR_MoM_Actual_sub_US_NF_HWR_MoM_Revised',
       'US_NF_HWR_MoM_Actual_sub_US_NF_HWR_MoM_Prior', '3s_60min'],
      dtype='object')

In [58]:
import numpy as np

for column in df.columns.tolist():
    try:
        df[column] = df[column].astype(np.float32)
    except:
        pass
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 155 entries, 2023-05-05 20:30:00 to 2010-01-08 21:30:00
Data columns (total 24 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   US_NF_EC_MoM_Actual                             155 non-null    float32
 1   US_NF_EC_MoM_Srv_Med                            155 non-null    float32
 2   US_NF_EC_MoM_Prior                              155 non-null    float32
 3   US_NF_EC_MoM_Revised                            155 non-null    float32
 4   US_NF_UER_MoM_Actual                            155 non-null    float32
 5   US_NF_UER_MoM_Srv_Med                           155 non-null    float32
 6   US_NF_UER_MoM_Prior                             155 non-null    float32
 7   US_NF_UER_MoM_Revised                           155 non-null    float32
 8   US_NF_HWR_MoM_Actual                            155 non-null    float32
 9   US_NF_HWR_MoM_

In [59]:
Data = TabularDataset(df.drop(["3s_1min", "3s_3min", "3s_5min", "3s_15min", "3s_30min", "3s_60min"], axis=1))

# Data=TabularDataset(df[['now_1', 'predict_1', 'previous_1', 'correction_1', 'now_2',
#        'predict_2', 'previous_2', 'correction_2','now_predict_sub_1', 'now_previous_sub_1',
#        'now_correction_sub_1', 'now_predict_sub_2', 'now_previous_sub_2',
#        'now_correction_sub_2',"label"]])


In [60]:
#TODO 根据不同分钟级label得出不同
minute_list = [1, 3, 5, 15, 30, 60]

In [61]:
df

,US_NF_EC_MoM_Actual,US_NF_EC_MoM_Srv_Med,US_NF_EC_MoM_Prior,US_NF_EC_MoM_Revised,US_NF_UER_MoM_Actual,US_NF_UER_MoM_Srv_Med,US_NF_UER_MoM_Prior,US_NF_UER_MoM_Revised,US_NF_HWR_MoM_Actual,US_NF_HWR_MoM_Srv_Med,...,3s_5min,3s_15min,3s_30min,US_NF_EC_MoM_Actual_sub_US_NF_EC_MoM_Srv_Med,US_NF_EC_MoM_Actual_sub_US_NF_EC_MoM_Revised,US_NF_EC_MoM_Actual_sub_US_NF_EC_MoM_Prior,US_NF_HWR_MoM_Actual_sub_US_NF_HWR_MoM_Srv_Med,US_NF_HWR_MoM_Actual_sub_US_NF_HWR_MoM_Revised,US_NF_HWR_MoM_Actual_sub_US_NF_HWR_MoM_Prior,3s_60min
datetime,,,,,,,,,,,,,,,,,,,,,
2023-05-05 20:30:00,253.0,236.0,236.0,165.0,3.4,3.5,3.5,3.5,0.5,0.3,...,1.0,-1.0,-1.0,17.0,88.0,17.0,0.2,0.2,0.2,-1.0
2023-03-10 21:30:00,311.0,205.0,517.0,504.0,3.6,3.4,3.4,3.4,0.2,0.3,...,1.0,1.0,1.0,106.0,-193.0,-206.0,-0.1,-0.1,-0.1,1.0
2023-02-03 21:30:00,517.0,185.0,223.0,260.0,3.4,3.6,3.5,3.5,0.3,0.3,...,-1.0,-1.0,-1.0,332.0,257.0,294.0,0.0,-0.1,0.0,-1.0
2023-01-06 21:30:00,223.0,200.0,263.0,256.0,3.5,3.7,3.7,3.6,0.3,0.4,...,-1.0,1.0,1.0,23.0,-33.0,-40.0,-0.1,-0.1,-0.3,1.0
2022-12-02 21:30:00,263.0,200.0,261.0,284.0,3.7,3.7,3.7,3.7,0.6,0.3,...,-1.0,-1.0,-1.0,63.0,-21.0,2.0,0.3,0.1,0.2,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-06-04 20:30:00,431.0,500.0,290.0,290.0,9.7,9.8,9.9,9.9,0.3,0.1,...,1.0,-1.0,1.0,-69.0,141.0,141.0,0.2,0.2,0.3,1.0
2010-05-07 20:30:00,290.0,187.0,162.0,230.0,9.9,9.7,9.7,9.7,0.0,0.2,...,1.0,1.0,1.0,103.0,60.0,128.0,-0.2,0.1,0.1,1.0
2010-03-05 21:30:00,-36.0,-55.0,-20.0,-26.0,9.7,9.8,9.7,9.7,0.1,0.2,...,-1.0,-1.0,-1.0,19.0,-10.0,-16.0,-0.1,-0.2,-0.2,1.0


# autogluon训练

In [62]:
from sklearn.metrics import classification_report
import numpy as np

for minute in minute_list[:]:
    minute_label_df = pd.concat([Data, df["3s_{}min".format(minute)]], axis=1)
    from model_datetime_utils import data_2_float

    data_2_float(minute_label_df)
    train_data = TabularDataset(minute_label_df)[fence:]  # Train
    test_data = TabularDataset(minute_label_df)[:fence]  # Test
    print(train_data.shape, test_data.shape)
    save_path = '{}_3s_{}'.format(model_name, minute)  # specifies folder to store trained models
    label = '3s_{}min'.format(minute)
    predictor = TabularPredictor(label=label, eval_metric='acc', problem_type="binary", path=save_path)
    predictor.fit(train_data, presets='best_quality')
    predictor = TabularPredictor.load(save_path)
    score = predictor.leaderboard(test_data, silent=True, extra_metrics=['roc_auc', 'mcc'])
    print("当前label分钟级别", minute)
    print("--------------------------------")
    print(score)
    model = score.model[0]
    pred = predictor.predict(test_data, model=model)
    onnx_test = onnx_test.copy()
    onnx_test["Pred_label"] = pred
    onnx_test["True_label"] = test_data['3s_{}min'.format(minute)]
    onnx_test.to_csv(r"{}_3s_{}\onnx_test.csv".format(model_name, minute),index=False)
    print(model)
    from generate_developer_md import get_developer_doc
    import os

    predictor_features = predictor.features()
    total_string = get_developer_doc(
        predictor_features,
        model_name=model_name,
        currency=trading_pair,
        shape=[train_data.shape[0], test_data.shape[0]],  # 训练集测试集数量
        use_model=score["model"][0],
        score=round(100 * score["score_test"][0], 2),
    )
    with open(os.getcwd() + r"\{}_3s_{}".format(model_name, minute) + r"\开发者文档.md",
              "w",
              encoding="utf-8") as f:
        f.write(total_string)



Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=5, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "NF_3s_1\"
AutoGluon Version:  0.7.0
Python Version:     3.9.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Train Data Rows:    140
Train Data Columns: 18
Label Column: 3s_1min
Preprocessing data ...
Selected class <--> label mapping:  class 1 = 1.0, class 0 = -1.0
	Note: For your binary classification, AutoGluon arbitrarily selected which label-value represents positive (1.0) vs negative (-1.0) class.
	To explicitly set the positive_class, either rename classes to 1 and 0, or specify positive_class in Predictor init.
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    4219.18 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each 

<class 'autogluon.core.dataset.TabularDataset'>
Index: 155 entries, 2023-05-05 20:30:00 to 2010-01-08 21:30:00
Data columns (total 19 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   US_NF_EC_MoM_Actual                             155 non-null    float32
 1   US_NF_EC_MoM_Srv_Med                            155 non-null    float32
 2   US_NF_EC_MoM_Prior                              155 non-null    float32
 3   US_NF_EC_MoM_Revised                            155 non-null    float32
 4   US_NF_UER_MoM_Actual                            155 non-null    float32
 5   US_NF_UER_MoM_Srv_Med                           155 non-null    float32
 6   US_NF_UER_MoM_Prior                             155 non-null    float32
 7   US_NF_UER_MoM_Revised                           155 non-null    float32
 8   US_NF_HWR_MoM_Actual                            155 non-null    float32
 9   US_N

	0.7143	 = Validation score   (accuracy)
	1.2s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.7143	 = Validation score   (accuracy)
	1.26s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L1 ...
	0.6643	 = Validation score   (accuracy)
	0.32s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L1 ...
	0.6571	 = Validation score   (accuracy)
	0.51s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.7357	 = Validation score   (accuracy)
	1.76s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini_BAG_L1 ...
	0.6286	 = Validation score   (accuracy)
	0.31s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: ExtraTreesEntr_BAG_L1 ..

当前label分钟级别 1
--------------------------------
                      model  score_test   roc_auc       mcc  score_val  \
0     KNeighborsDist_BAG_L1    0.600000  0.678571  0.218218   0.628571   
1     KNeighborsUnif_BAG_L1    0.600000  0.660714  0.218218   0.621429   
2         LightGBMXT_BAG_L1    0.533333  0.571429  0.250000   0.714286   
3     ExtraTreesEntr_BAG_L1    0.533333  0.625000  0.250000   0.657143   
4   RandomForestEntr_BAG_L1    0.533333  0.455357  0.250000   0.657143   
5   RandomForestGini_BAG_L1    0.533333  0.482143  0.250000   0.664286   
6           CatBoost_BAG_L1    0.466667  0.589286 -0.026207   0.735714   
7           LightGBM_BAG_L1    0.466667  0.500000 -0.040291   0.714286   
8      LightGBMLarge_BAG_L1    0.466667  0.410714 -0.040291   0.757143   
9       WeightedEnsemble_L2    0.466667  0.410714 -0.040291   0.757143   
10    ExtraTreesGini_BAG_L1    0.466667  0.669643 -0.026207   0.628571   
11   NeuralNetFastAI_BAG_L1    0.466667  0.803571  0.000000   0.7

	0.6929	 = Validation score   (accuracy)
	1.17s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.6929	 = Validation score   (accuracy)
	1.37s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L1 ...
	0.6643	 = Validation score   (accuracy)
	0.35s	 = Training   runtime
	0.06s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L1 ...
	0.6857	 = Validation score   (accuracy)
	0.36s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.7	 = Validation score   (accuracy)
	1.85s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini_BAG_L1 ...
	0.6571	 = Validation score   (accuracy)
	0.38s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: ExtraTreesEntr_BAG_L1 ...
	

...

当前label分钟级别 15
--------------------------------
                      model  score_test   roc_auc       mcc  score_val  \
0           CatBoost_BAG_L1    0.733333  0.409091  0.000000   0.657143   
1       WeightedEnsemble_L2    0.733333  0.409091  0.000000   0.657143   
2     ExtraTreesGini_BAG_L1    0.733333  0.488636  0.000000   0.464286   
3    NeuralNetFastAI_BAG_L1    0.733333  0.500000  0.000000   0.592857   
4         LightGBMXT_BAG_L1    0.666667  0.363636 -0.161165   0.614286   
5     ExtraTreesEntr_BAG_L1    0.666667  0.477273 -0.161165   0.485714   
6   RandomForestGini_BAG_L1    0.666667  0.113636 -0.161165   0.578571   
7           LightGBM_BAG_L1    0.533333  0.340909 -0.301511   0.628571   
8            XGBoost_BAG_L1    0.533333  0.477273 -0.301511   0.614286   
9     KNeighborsUnif_BAG_L1    0.466667  0.261364 -0.184637   0.550000   
10    KNeighborsDist_BAG_L1    0.466667  0.250000 -0.184637   0.564286   
11  RandomForestEntr_BAG_L1    0.466667  0.170455 -0.363636   0.

	0.5357	 = Validation score   (accuracy)
	0.01s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMXT_BAG_L1 ...
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.5571	 = Validation score   (accuracy)
	1.28s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.6	 = Validation score   (accuracy)
	1.19s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L1 ...
	0.4786	 = Validation score   (accuracy)
	0.35s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L1 ...
	0.4643	 = Validation score   (accuracy)
	0.69s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.5786	 = Validation score   (accuracy)
	1.49s	 = Trainin

当前label分钟级别 30
--------------------------------
                      model  score_test   roc_auc       mcc  score_val  \
0           CatBoost_BAG_L1    0.800000  0.568182  0.452267   0.578571   
1            XGBoost_BAG_L1    0.733333  0.681818  0.206959   0.542857   
2           LightGBM_BAG_L1    0.666667  0.795455  0.492366   0.600000   
3         LightGBMXT_BAG_L1    0.666667  0.681818  0.213201   0.557143   
4     ExtraTreesGini_BAG_L1    0.666667  0.477273  0.075378   0.478571   
5   RandomForestEntr_BAG_L1    0.666667  0.420455  0.075378   0.464286   
6   RandomForestGini_BAG_L1    0.666667  0.488636  0.075378   0.478571   
7      LightGBMLarge_BAG_L1    0.600000  0.272727 -0.236525   0.621429   
8    NeuralNetFastAI_BAG_L1    0.600000  0.272727 -0.236525   0.628571   
9     ExtraTreesEntr_BAG_L1    0.533333  0.511364 -0.106600   0.428571   
10      WeightedEnsemble_L2    0.533333  0.295455 -0.301511   0.642857   
11    KNeighborsDist_BAG_L1    0.466667  0.431818 -0.040291   0.

	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.6429	 = Validation score   (accuracy)
	1.36s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ...
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.6643	 = Validation score   (accuracy)
	1.43s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: RandomForestGini_BAG_L1 ...
	0.5857	 = Validation score   (accuracy)
	0.4s	 = Training   runtime
	0.08s	 = Validation runtime
Fitting model: RandomForestEntr_BAG_L1 ...
	0.5857	 = Validation score   (accuracy)
	0.43s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ...
	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy
	0.7286	 = Validation score   (accuracy)
	1.47s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: ExtraTreesGini_BAG_L1 ...
	0.5786	 = Validation score   (accuracy)
	0.31s	 = Tr

当前label分钟级别 60
--------------------------------
                      model  score_test   roc_auc       mcc  score_val  \
0           LightGBM_BAG_L1    0.666667  0.740741  0.388889   0.664286   
1     ExtraTreesEntr_BAG_L1    0.666667  0.805556  0.492366   0.592857   
2         LightGBMXT_BAG_L1    0.600000  0.814815  0.288675   0.642857   
3           CatBoost_BAG_L1    0.600000  0.814815  0.408248   0.728571   
4      LightGBMLarge_BAG_L1    0.600000  0.722222  0.408248   0.657143   
5     ExtraTreesGini_BAG_L1    0.600000  0.796296  0.408248   0.578571   
6            XGBoost_BAG_L1    0.600000  0.722222  0.408248   0.628571   
7     KNeighborsUnif_BAG_L1    0.533333  0.648148  0.184637   0.528571   
8     KNeighborsDist_BAG_L1    0.533333  0.703704  0.184637   0.535714   
9   RandomForestEntr_BAG_L1    0.533333  0.925926  0.320256   0.585714   
10    NeuralNetTorch_BAG_L1    0.533333  0.870370  0.320256   0.678571   
11      WeightedEnsemble_L2    0.533333  0.851852  0.320256   0.

# 回测模板生成